In [3]:
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()

input_text_list

['나는', '최근', '파리', '여행을', '다녀왔다']

In [4]:
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}

print(str2idx)
print(idx2str)

{'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
{0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}


In [5]:
input_ids = [str2idx[word] for word in input_text_list]
input_ids

[0, 1, 2, 3, 4]

In [6]:
import torch
import torch.nn as nn

In [7]:
embedding_dims = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim=embedding_dims)

In [8]:
input_embeddings = embed_layer(torch.tensor(input_ids))
input_embeddings = input_embeddings.unsqueeze(0)
input_embeddings.shape

torch.Size([1, 5, 16])

In [9]:
embedding_dim = 16
max_position = 12
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids))
token_embeddings = token_embeddings.unsqueeze(0)
input_embeddings = token_embeddings + position_encodings

input_embeddings.shape

torch.Size([1, 5, 16])

In [10]:
head_dim = 16

weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

querys = weight_q(input_embeddings)
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)

In [11]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1)
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return weights @ values

In [12]:
print("원본 입력 형태 : ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 입력 형태 : ", after_attention_embeddings.shape)

원본 입력 형태 :  torch.Size([1, 5, 16])
어텐션 적용 후 입력 형태 :  torch.Size([1, 5, 16])


In [13]:
class AttentionHead(nn.Module):

    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        self.weight_v = nn.Linear(token_embed_dim, head_dim)
        self.is_causal = is_causal

    def forward(self, querys, keys, values):
        output = compute_attention(self.weight_q(querys), self.weight_k(keys), self.weight_v(values), self.is_causal)
        return output
        


In [14]:
import torch.nn as nn
class MultiHeadAttention(nn.Module):


    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()

        self.n_head = n_head
        self.is_causal = is_causal

        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)

    def forward(self, querys, keys, values):
        # B: batch size (배치 크기)
        # T: sequence length (시퀀스/토큰 길이)
        # C: embedding dimension (임베딩 차원)
        B, T, C = querys.size()

        # 1. Linear 변환으로 d_model 차원으로 변환 (B, T, d_model)
        # 2. view로 (B, T, n_head, head_dim)으로 나누기
        #    - d_model을 n_head와 head_dim(d_model/n_head)로 분할
        # 3. transpose로 (B, n_head, T, head_dim)으로 변환
        #    - head 차원을 앞으로 가져와서 각 head가 독립적인 (T, head_dim) 행렬을 볼 수 있게 됨
        #    - compute_attention은 (B, n_head, T, head_dim) 형태의 입력을 받아
        #      각 head별로 독립적으로 attention 연산을 수행할 수 있음
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        # 4. 각 헤드에서 어텐션 계산
        # compute_attention을 통해 각 헤드별로 어텐션 스코어를 계산
        # attention shape: (batch, n_head, seq_len, head_dim)
        attention = compute_attention(querys, keys, values, self.is_causal)
        
        # 5. 헤드 차원을 다시 원래대로 되돌리는 과정
        # transpose(1,2)로 (batch, seq_len, n_head, head_dim) 형태로 변환
        # contiguous()는 메모리상 연속적으로 배치하여 view 연산을 가능하게 함
        # view를 통해 (batch, seq_len, d_model) 형태로 모든 헤드의 결과를 하나로 합침
        output = attention.transpose(1, 2).contiguous().view(B, T, C)

        # 5. 최종적으로 모든 헤드의 결과를 결합하여 최종 출력 생성
        output = self.concat_linear(output)

        return output

In [ ]:
n_head = 4
mh_attention = MultiHeadAttention(token_embed_dim=16, d_model=16, n_head=4)
after_mh_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)

print("어텐션 적용 후 입력 형태 : ", after_mh_attention_embeddings.shape)